<h3><font color='red'>What is Run-Length encoding(RLE)?</h3>
<h4><font color='green'>Run-length encoding (RLE) is a form of lossless data compression in which runs of data (sequences in which the same data value occurs in many consecutive data elements) are stored as a single data value and count, rather than as the original run.</h4>

<img src='https://www.dspguide.com/graphics/F_27_1.gif' width='1000' height='300'>

<h3><font color='red'>Example</h3>
<h4><font color='green'>For example, if the input string is “wwwwaaad”, then the function should return “w4a3d1”<br>Here w appears 4, a appears 3 times and d appears 1 time so we take each characters and then put their number of appearances after them</h4>

<h2><font color='red'>Please don't forget to upvote </h2>

In [ ]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train_fol = '../input/sartorius-cell-instance-segmentation/train'
test_fol = '../input/sartorius-cell-instance-segmentation/test'
train = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')

In [ ]:
#ref:https://www.kaggle.com/paulorzp/run-length-encode-and-decode.
def encode_mask_to_rle(mask):
    '''
    mask: numpy array binary mask 
    1 - mask 
    0 - background
    Returns encoded run length 
    '''
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

example = np.array([1,1,1,1,0,0,0,1,0,1,1,0,0,0,1,1,1])
rle_encoded = encode_mask_to_rle(example)
print(rle_encoded)

<h4><font color='green'>Here we encoded a example mask to run-length string. We tried to preserve the places of value 1.<br>
Since it's a instance segmentation competition sothere is no class for segmented masks and all mask values will be 1 for stuff and 0 for background.
</h4>

<h3><font color='red'>Run-Length Decoding</h3>
<h4><font color='green'>In this competition there are multiple rows for each image id and in each row there is a segmentation mask for each neuronal cell</h4> 

In [ ]:
def decode_rle_to_mask(rle, height, width, viz=False):
    '''
    rle : run-length as string formated (start value, count)
    height : height of the mask 
    width : width of the mask
    returns binary mask
    '''
    rle = np.array(rle.split(' ')).reshape(-1, 2)
    mask = np.zeros((height*width, 1, 3))
    if viz:
        color = np.random.rand(3)
    else:
        color = [1,1,1]
    for i in rle:
        mask[int(i[0]):int(i[0])+int(i[1]), :, :] = color

    return mask.reshape(height, width, 3)

<h3>Example</h3>

In [ ]:
example_id = '0030fd0e6378' # change the example id for different images
image = cv2.imread(os.path.join(train_fol, example_id)+'.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = (image-np.min(image))/(np.max(image)-np.min(image))
print(image.shape)
print("This is the image we are going to use, id:-", example_id)
plt.imshow(image)
plt.show()

<h4><font color='green'>lets try to convert one annotation to its respective mask</h4>

In [ ]:
example_annot = train[train.id==example_id].annotation.values[0]
print("The Annotation : ",example_annot)
print("Now let's convert it to mask and view it")
mask = decode_rle_to_mask(example_annot, image.shape[0], image.shape[1], viz=False)
plt.imshow(mask)
plt.show()

<h4><font color='green'>Lets try to do it on all the annotations of that id and then add them together!</h4>

In [ ]:
example_id_annotations = train[train.id==example_id].annotation.values
masks = np.zeros(image.shape)
for each_cell_annot in example_id_annotations:
    mask = decode_rle_to_mask(each_cell_annot, image.shape[0], image.shape[1], viz=True)
    masks+=mask

plt.imshow(image)
plt.show()
plt.imshow(masks)
plt.show()
t = image+masks
plt.imshow(t)
plt.show()

<h4><font color='green'>Here we decoded run-length to mask and added them together to get a mask and applied it over the image</h4> 

In [ ]:
example_id_annotations = train[train.id==example_id].annotation.values
masks = np.zeros(image.shape)
for each_cell_annot in example_id_annotations:
    mask = decode_rle_to_mask(each_cell_annot, image.shape[0], image.shape[1], viz=False)
    masks+=mask

plt.imshow(image)
plt.show()
plt.imshow(masks)
plt.show()
t = image+masks
plt.imshow(t)
plt.show()

In [ ]:
def show_img_mask(Id, motiv='train'):
    if motiv=='train':
        fol = train_fol
    if motiv=='test':
        fol = test_fol
    image = cv2.imread(os.path.join(fol, Id)+'.png')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = (image-np.min(image))/(np.max(image)-np.min(image))
    print("Image id:-", Id)
    plt.imshow(image)
    plt.show()
    example_id_annotations = train[train.id==example_id].annotation.values
    masks = np.zeros(image.shape)
    for each_cell_annot in example_id_annotations:
        mask = decode_rle_to_mask(each_cell_annot, image.shape[0], image.shape[1], viz=True)
        masks+=mask
    plt.imshow(masks)
    plt.show()

In [ ]:
show_img_mask('0030fd0e6378', motiv='train')

In [ ]:
show_img_mask('7ae19de7bc2a', motiv='test')

<h3><font color='brown'>Hope You Liked it :D</h3>